<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.875 · Deep Learning · PEC1</p>
<p style="margin: 0; text-align:right;">2018-2 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# PEC 1: Redes neuronales completamente conectadas

En esta práctica implementaremos una red neuronal completamente conectada de dos formas diferentes: 

<ol start="1">
  <li>Partiendo de cero utilizando únicamente la librería numpy</li>
  <li>Utilizando la librería Keras y TensorFlow</li>
</ol>

Posteriormente utilizaremos las dos implementaciones para entrenar dos redes neuronales iguales en un conjunto de datos y compararemos el rendimiento.

**Importante: Cada uno de los ejercicios puede suponer varios minutos de ejecución, por lo que la entrega debe hacerse en formato notebook y en formato html donde se vea el código y los resultados y comentarios de cada ejercicio. Para exportar el notebook a html puede hacerse desde el menú File $\to$ Download as $\to$ HTML.**

## 0. Carga de datos

El siguiente código carga los paquetes necesarios para la práctica y además lee los datos que utilizaremos para entrenar la red neuronal.

In [1]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

with open("data.pickle", "rb") as f:
    data = pickle.load(f)

features = data["features"]
labels = data["labels"]

train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.2)

## 1. Redes neuronales utilizando numpy 

A continuación implementaremos todas las funciones necesarias para entrenar una red neuronal completamente conectada utilizando únicamente la librería numpy. El objetivo es poder entrenar una red neuronal con cualquier número de capas en la cual la última capa tendrá una única neurona con función de activación sigmoid y las demás capas cualquier número de neuronas con función de activación relu.

La siguiente figura muestra un diagrama de como implementaremos el proceso de entrenamiento de la red neuronal:

<img src="diag.png" alt="Diagrama del entrenamiento de la red neuronal" style="height: 550px;"/>

El desarrollo está estructurado en funciones básicas que se componen según el siguiente esquema:

- L_layer_model
  - initialize_parameters
  - L_model_forward
    - linear_activation_forward
      - linear_forward
      - sigmoid
      - relu
  - compute_cost
  - L_model_backward
    - linear_activation_backward
      - linear_backward
      - sigmoid_backward
      - relu_backward
  - update_parameters
- accuracy

**Notación**:
- Denotamos $L$ el número de capas de la red neuronal.
- La matriz de pesos que conecta una capa con la siguiente la denotamos con la letra $W$, mientras que el vector de bias lo denotamos con la letra $b$.
- Superíndice $[l]$ denota una cantidad asociada con la capa número $l$. 
    - Ejemplo: $a^{[L]}$ denota la salida de la capa número $L$.
    - Ejemplo: Las variables $W^{[L]}$ y $b^{[L]}$ denotan la matriz de pesos y el vector de bias que conectan la capa $L-1$ con la capa $L$ respectivamente.
- Superíndice $(i)$ denota una cantidad asociada con el ejemplo $i$-ésimo. 
    - Ejemplo: $x^{(i)}$ es el ejemplo del conjunto de entrenamiento número $i$.

### 1.1 Inicialización de parámetros

El primer paso para entrenar una red neuronal consiste en inicializar de forma aleatoria los parámetros del modelo.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Inicializar las matrices de parámetros y los vectores de bias. Las matrices de pesos se deben inicializar utilizando la distribución normal y los vectores de bias se deben inicializar con ceros. Para las matrices de pesos podéis utilizar 0.1*np.random.randn(shape) indicando el tamaño correcto.
</div>

In [2]:
def initialize_parameters(layer_dims):
    """
    Argumentos:
    layer_dims -- lista que contiene las dimensiones de cada capa de la red
    
    Devuelve:
    parameters -- diccionario que contiene los parametros "W1", "b1", ..., "WL", "bL":
                    Wl -- matriz de pesos de tamaño (layer_dims[l], layer_dims[l-1])
                    bl -- vector de bias de tamaño (layer_dims[l], 1)
    """
    
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        #TODO: REMOVE 2 LINES
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
    
    return parameters

### 1.2 Propagación hacia delante

En una capa concreta de una red neuronal, las entradas de las neuronas se combinan de forma lineal antes de pasar por la función de activación según la siguiente fórmula:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$$

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Calcular la combinación lineal de las entradas a una capa de la red neuronal.
</div>

In [3]:
def linear_forward(A, W, b):
    """
    Implementa la parte lineal de la propagación hacia delante de una capa

    Argumentos:
    A -- salida de la capa anterior (o datos de entrada): (número de neuronas de la capa anterior, número de ejemplos)
    W -- matriz de pesos: (número de neuronas de la capa actual, número de neuronas de la capa anterior)
    b -- vector de bias: (número de neuronas de la capa actual, 1)

    Devuelve:
    Z -- la entrada a la función de activación
    cache -- una tripleta que contiene "A", "W" y "b", utilizada después para la propagación hacia atrás
    """
    
    Z = np.dot(W, A) + b
    
    #TODO: REMOVE 1 LINE
    assert(Z.shape == (W.shape[0], A.shape[1]))
    
    cache = (A, W, b)
    
    return Z, cache

Una vez se ha calculado la combinación lineal de las entradas de una capa se debe aplicar una función de activación no lineal antes de enviar las salidas a la siguiente capa. Si denotamos $g$ la función de activación (en nuestro caso relu o sigmoid), tenemos la siguiente fórmula:

$$A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} + b^{[l]})$$

A continuación definimos las funciones de activación que utilizaremos en la red neuronal.

In [4]:
def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    cache = Z
    return A, cache

def relu(Z):
    A = np.maximum(0, Z)
    #TODO: REMOVE 1 LINE
    assert(A.shape == Z.shape)
    cache = Z
    return A, cache

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Calcular la combinación lineal de las entradas utilizando la función implementada anteriormente y aplicar la función de activación no lineal que corresponda.
</div>

In [5]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implementa la propagación hacia delante de una capa incluyendo la función de activación

    Argumentos:
    A_prev -- salida de la capa anterior (o datos de entrada): 
                (número de neuronas de la capa anterior, número de ejemplos)
    W -- matriz de pesos: (número de neuronas de la capa actual, número de neuronas de la capa anterior)
    b -- vector de bias: (número de neuronas de la capa actual, 1)
    activation -- el nombre de la función de activación a utilizar en la capa: "sigmoid" o "relu"

    Devuelve:
    A -- la salida de la capa después de aplicar la función de activación
    cache -- una dupla que contiene "linear_cache" y "activation_cache", utilizada después para la propagación hacia atrás
    """
    
    Z, linear_cache = linear_forward(A_prev, W, b)
    
    if activation == "sigmoid":    
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        A, activation_cache = relu(Z)
    
    # TODO: REMOVE 1 LINE
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

Dados los datos de entrada, la salida de la red neuronal se calcula aplicando diferentes capas una detrás de otra. Si denotamos la última capa como $L$, la salida de la red neuronal se corresponde con la salida de la última capa $A^{[L]}$.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Calcular la salida de la red neuronal aplicando $L-1$ capas con función de activación relu y una última capa con función de activación sigmoid.
</div>

In [6]:
def L_model_forward(X, parameters):
    """
    Implementa la propagación hacia delante de la red neuronal completa
    
    Argumentos:
    X -- datos: matriz de tamaño (número de variables, número de ejemplos)
    parameters -- salida de la función initialize_parameters()
    
    Devuelve:
    AL -- salida de la red neuronal
    caches -- lista de caches que contiene todas las caches de la función linear_activation_forward(), las caches
                indexadas de 0 a L-2 corresponden a las caches de la función de activación relu y la cache indexada
                como L-1 corresponde a la cache de la función de activación sigmoid
    """

    caches = []
    A = X
    L = len(parameters) // 2
    
    # Implementa primero las L-1 capas con función de activación relu
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, 
                                             parameters['W' + str(l)], 
                                             parameters['b' + str(l)], 
                                             activation='relu')
        caches.append(cache)
    
    # Implementa la última capa con función de activación sigmoid
    AL, cache = linear_activation_forward(A, 
                                          parameters['W' + str(L)], 
                                          parameters['b' + str(L)], 
                                          activation='sigmoid')
    caches.append(cache)
    
    return AL, caches

### 1.3 Función de coste

Una vez hemos obtenido la salida de la red neuronal podemos obtener un valor que mida el rendimiento de la red neuronal utilizando una función de coste $\mathcal{L}$. En nuestro caso utilizaremos la función de coste log-loss, que viene definida por la siguiente fórmula:

$$\mathcal{L} = -\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right))$$

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Calcular el valor de la función de coste log-loss dada la salida de la red neuronal junto con las etiquetas correctas.
</div>

In [7]:
def compute_cost(AL, Y):
    """
    Calcula la función de coste

    Argumentos:
    AL -- vector que contiene la salida de la red, corresponde a las probabilidades que predice la red neuronal
            para cada ejemplo: (1, número de ejemplos)
    Y -- vector con las etiquetas correctas para los datos de entrada a la red: (1, número de ejemplos)

    Devuelve:
    cost -- valor de la función de coste log-loss
    """
    
    m = Y.shape[1]

    cost = (-1 / m) * np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1 - Y, np.log(1 - AL)))
    cost = np.squeeze(cost)
    
    return cost

### 1.4 Propagación hacia atrás

Para entrenar una red neuronal es necesario calcular el gradiente de la función de coste repescto a los parámetros de la red, para lo cual utilizaremos la propagación hacia atrás. La propagación hacia atrás consiste en aplicar la regla de la cadena para calcular el gradiente de la función de coste paso a paso en cada capa.

Para aplicar la regla de la cadena en la parte lineal de la neurona, supongamos que ya hemos calculado la derivada $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$. Entonces, para calcular las derivadas $(dW^{[l]}, db^{[l]}, dA^{[l-1]})$ podemos utilizar las siguientes fórmulas:

$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]}$$

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Calcular las derivadas de la parte lineal para una sola capa.
</div>

In [8]:
def linear_backward(dZ, cache):
    """
    Implementa la parte lineal de la propagación hacia atrás para una única capa

    Argumentos:
    dZ -- derivada de la función de coste con respecto a la salida lineal de la capa actual
    cache -- tripleta que contiene los valores (A_prev, W, b), provinientes de la función linear_forward

    Devuelve:
    dA_prev -- derivada de la función de coste con respecto a la salida de la capa anterior (l-1): 
                tiene el mismo tamaño que A_prev
    dW -- derivada de la función de coste con respecto a la matriz de pesos W de la capa actual (l):
                tiene el mismo tamaño que W
    db -- derivada de la función de coste con respecto al vector de bias b de la capa actual (l):
                tiene el mismo tamaño que b
    """
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = 1 / m * dZ @ A_prev.T
    db = 1 / m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = W.T @ dZ
    #dW = 1 / m * np.dot(dZ, A_prev.T)
    #db = 1 / m * np.sum(dZ, axis = 1, keepdims = True)
    #dA_prev = np.dot(W.T, dZ)
    
    # TODO: REMOVE 3 LINES
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)

    return dA_prev, dW, db

El siguiente paso consiste en aplicar la regla de la cadena a la parte no lineal de las neuronas, es decir, a las funciones de activación. Para esto, si denotamos $g$ la función de activación, podemos utilizar la siguiente fórmula:

$$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]})$$

Donde $*$ indica el producto componente a componente.

A continuación calculamos las derivadas de las funciones de activación que utilizamos en la red neuronal.

In [9]:
def sigmoid_backward(dA, cache):
    Z = cache
    s = 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1 - s)
    return dZ

def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Combinar el cálculo de la derivada de las funciones de activación con la derivada de la parte lineal para obtener, a partir de la derivada de la función de coste respecto la activación de una capa, la derivada de la función de coste respecto a los parámetros de la capa y respecto a las activaciones de la capa anterior.
</div>

In [10]:
def linear_activation_backward(dA, cache, activation):
    """
    Implementa la propagación hacia atrás de una única capa incluyendo la función de activación
    
    Argumentos:
    dA -- derivada de la función de coste con respecto a la salida de la capa actual (l)
    cache -- dupla que contiene "linear_cache" y "activation_cache", provinientes de la función linear_activation_forward
    activation -- el nombre de la función de activación utilizada en la capa actual (l): "sigmoid" o "relu"
    
    Devuelve:
    dA_prev -- derivada de la función de coste con respecto a la salida de la capa anterior (l-1):
                tiene el mismo tamaño que A_prev
    dW -- derivada de la función de coste con respecto a la matriz de pesos W de la capa actual (l):
                tiene el mismo tamaño que W
    db -- derivada de la función de coste con respecto al vector de bias b de la capa actual (l):
                tiene el mismo tamaño que b
    """
    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

Por último, es posible calcular la derivada de la función de coste respecto a cualquiera de los parámetros aplicando las funciones recién implementadas empezando por la última capa. Observemos que para inicializar la propagación hacia atrás es necesario calcular primero el valor de $\frac{\partial \mathcal{L}}{\partial A^{[L]}}$.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Aplicar la propagación hacia atrás para calcular el gradiente de la función de coste. Observad que el valor de $\frac{\partial \mathcal{L}}{\partial A^{[L]}}$ viene calculado en la variable dAL y que la última capa tiene función de activación sigmoid mientras que todas las demás tienen función de activación relu.
</div>

In [11]:
def L_model_backward(AL, Y, caches):
    """
    Implementa la propagación hacia atrás de la red neuronal completa
    
    Argumentos:
    AL -- salida de la red neuronal, proviene de la función L_model_forward
    Y -- vector con las etiquetas correctas para cada ejemplo del conjunto de datos: (1, número de ejemplos)
    caches -- lista de caches que contiene todas las caches de la función linear_activation_forward(), las caches
                indexadas de 0 a L-2 corresponden a las caches de la función de activación relu y la cache indexada
                como L-1 corresponde a la cache de la función de activación sigmoid
    
    Devuelve:
    grads -- Un diccionario con las derivadas de la función de coste respecto de cada variable:
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """

    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    # Inicialización de la propagación hacia atrás
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Gradiente de la última capa
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, 'sigmoid')
    
    # Gradiente de las capas restantes
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, 'relu')
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

### 1.5 Actualización de parámetros

Una vez disponemos del gradiente de la función de coste podemos utilizar el método del descenso del gradiente para actualizar los parámetros de la red neuronal. Si denotamos $\alpha$ la velocidad de aprendizaje, las fórmulas para aplicar un paso del descenso del gradiente son:

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]}$$

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Actualizar los parámetros de la red neuronal aplicando un paso del descenso del gradiente.
</div>

In [12]:
def update_parameters(parameters, grads, learning_rate):
    """
    Actualiza los parámetros utilizando el descenso del gradiente
    
    Argumentos:
    parameters -- diccionario que contiene los parámetros de la red neuronal
    grads -- diccionario con las derivadas de la función de coste respecto a cada parámetro,
                corresponde a la salida de la función L_model_backward
    
    Devuelve:
    parameters -- diccionario con los parámetros actualizados:
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2

    for l in range(L):
        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]

    return parameters

Con todo esto es posible entrenar la red neuronal combinando las funciones definidas anteriormente para aplicar diversas iteraciones del descenso del gradiente e ir actualizando los parámetros de la red de forma reiterada.

El siguiente código muestra cómo entrenar la red neuronal que hemos construido utilizando únicamente la librería numpy.

In [13]:
def L_layer_model(X, Y, layers_dims, learning_rate, num_iterations, print_cost):
    """
    Implementa una red neuronal de L capas donde las L-1 primeras capas tienen función de activación relu y 
    la última capa tiene función de activación sigmoid.
    
    Argumentos:
    X -- datos: matriz de tamaño (número de variables, número de ejemplos)
    Y -- vector con las etiquetas correctas para cada ejemplo del conjunto de datos: (1, número de ejemplos)
    layers_dims -- lista de longitud (número de capas + 1) que contiene el número de variables y el número 
                    de neuronas en cada capa, 
    learning_rate -- velocidad de aprendizaje para aplicar el método del descenso del gradiente
    num_iterations -- número de pasos para aplicar el descenso del gradiente
    print_cost -- si el valor es True, escribe el valor de la función de coste cada 10 iteraciones
    
    Devuelve:
    parameters -- parámetros ajustados de la red neuronal
    """
    
    # Inicialización de los parámetros
    parameters = initialize_parameters(layers_dims)
    
    for i in range(0, num_iterations):
        # Propagación hacia delante
        AL, caches = L_model_forward(X, parameters)
        
        # Cálculo de la función de coste
        cost = compute_cost(AL, Y)
    
        # Propagación hacia atrás
        grads = L_model_backward(AL, Y, caches)
 
        # Actualización de parámetros
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Escribe el valor de la función de coste cada 10 iteraciones
        if print_cost and i % 10 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    return parameters

## 2. Redes neuronales utilizando Keras

A continuación definiremos una red neuronal completamente conectada igual a la que hemos implementado anteriormente pero esta vez utilizando la librería Keras.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Ejercicio:</strong> Definir una red neuronal completamente conectada a partir de una lista que contiene el número de neuronas que debe tener cada capa de la red. Las primeras capas deben tener función de activación relu y la última capa debe tener función de activación sigmoid. Todas ellas tienen que tener kernel_initializer="random_normal" y bias_initializer="zeros".
</div>

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

def keras_model(layers_dims, learning_rate):
    """
    Crea, utilizando Keras, una red neuronal de L capas completamente conectadas donde las L-1 primeras capas
    tienen función de activación relu y la última capa tiene función de activación sigmoid.
    
    Argumentos:
    layers_dims -- lista de longitud (número de capas + 1) que contiene el número de variables y el número 
                    de neuronas en cada capa, 
    learning_rate -- velocidad de aprendizaje para aplicar el método del descenso del gradiente
    
    Devuelve:
    modelo -- objeto de Keras que representa la red neuronal
    """
    
    L = len(layers_dims)
    
    model = Sequential()
    
    # Añadir L-1 capas con función de activación relu y una última capa con función de activación sigmoid,
    # cada capa debe tener el número de neuronas indicado en la variable layers_dims, el tamaño de la capa
    # de entrada viene dado en layers_dims[0]
    
    for l in range(1, L-1):
        model.add(keras.layers.Dense(layers_dims[l], kernel_initializer='random_normal' ,
                bias_initializer='zeros', activation="relu"))

    
    model.add(keras.layers.Dense(layers_dims[L-1], kernel_initializer='random_normal' ,
                bias_initializer='zeros', activation="sigmoid"))
    
    
    model.compile(optimizer=SGD(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

Using TensorFlow backend.


## 3. Entrenamiento de la red neuronal

Con todas las funciones implementadas anteriormente es posible entrenar una red neuronal completamente conectada con cualquier número de capas y cualquier número de neuronas en cada capa.

A continuación definimos la estructura de capas que tendrá la red neuronal.

In [15]:
layers_dims = [100, 20, 5, 1]

Para entrenar la red neuronal que hemos construido únicamente utilizando numpy debemos ejecutar el siguiente código:

In [16]:
parameters = L_layer_model(train_x.T, train_y.reshape(1, -1), layers_dims=layers_dims, learning_rate=0.1, 
                           num_iterations=250, print_cost=True)

Cost after iteration 0: 0.693140
Cost after iteration 10: 0.693134
Cost after iteration 20: 0.693129
Cost after iteration 30: 0.693126
Cost after iteration 40: 0.693122
Cost after iteration 50: 0.693118
Cost after iteration 60: 0.693114
Cost after iteration 70: 0.693110
Cost after iteration 80: 0.693105
Cost after iteration 90: 0.693099
Cost after iteration 100: 0.693093
Cost after iteration 110: 0.693085
Cost after iteration 120: 0.693076
Cost after iteration 130: 0.693065
Cost after iteration 140: 0.693052
Cost after iteration 150: 0.693037
Cost after iteration 160: 0.693018
Cost after iteration 170: 0.692994
Cost after iteration 180: 0.692965
Cost after iteration 190: 0.692927
Cost after iteration 200: 0.692879
Cost after iteration 210: 0.692816
Cost after iteration 220: 0.692731
Cost after iteration 230: 0.692615
Cost after iteration 240: 0.692452


Para entrenar la red neuronal que hemos construido utilizando Keras debemos ejecutar el siguiente código:

In [17]:
model = keras_model(layers_dims = layers_dims, learning_rate = 0.1)
model.fit(train_x, train_y, epochs=250, batch_size=train_x.shape[0], verbose=2)

Epoch 1/250
 - 0s - loss: 0.6920 - acc: 0.5275
Epoch 2/250
 - 0s - loss: 0.6919 - acc: 0.5225
Epoch 3/250
 - 0s - loss: 0.6918 - acc: 0.5194
Epoch 4/250
 - 0s - loss: 0.6917 - acc: 0.5194
Epoch 5/250
 - 0s - loss: 0.6915 - acc: 0.5188
Epoch 6/250
 - 0s - loss: 0.6914 - acc: 0.5175
Epoch 7/250
 - 0s - loss: 0.6913 - acc: 0.5163
Epoch 8/250
 - 0s - loss: 0.6912 - acc: 0.5163
Epoch 9/250
 - 0s - loss: 0.6911 - acc: 0.5163
Epoch 10/250
 - 0s - loss: 0.6909 - acc: 0.5163
Epoch 11/250
 - 0s - loss: 0.6908 - acc: 0.5156
Epoch 12/250
 - 0s - loss: 0.6906 - acc: 0.5156
Epoch 13/250
 - 0s - loss: 0.6904 - acc: 0.5156
Epoch 14/250
 - 0s - loss: 0.6903 - acc: 0.5163
Epoch 15/250
 - 0s - loss: 0.6901 - acc: 0.5163
Epoch 16/250
 - 0s - loss: 0.6899 - acc: 0.5163
Epoch 17/250
 - 0s - loss: 0.6897 - acc: 0.5169
Epoch 18/250
 - 0s - loss: 0.6894 - acc: 0.5206
Epoch 19/250
 - 0s - loss: 0.6892 - acc: 0.5231
Epoch 20/250
 - 0s - loss: 0.6889 - acc: 0.5263
Epoch 21/250
 - 0s - loss: 0.6886 - acc: 0.5281
E

Epoch 171/250
 - 0s - loss: 0.1758 - acc: 0.9638
Epoch 172/250
 - 0s - loss: 0.1749 - acc: 0.9638
Epoch 173/250
 - 0s - loss: 0.1739 - acc: 0.9638
Epoch 174/250
 - 0s - loss: 0.1730 - acc: 0.9638
Epoch 175/250
 - 0s - loss: 0.1721 - acc: 0.9644
Epoch 176/250
 - 0s - loss: 0.1712 - acc: 0.9644
Epoch 177/250
 - 0s - loss: 0.1703 - acc: 0.9644
Epoch 178/250
 - 0s - loss: 0.1694 - acc: 0.9656
Epoch 179/250
 - 0s - loss: 0.1685 - acc: 0.9656
Epoch 180/250
 - 0s - loss: 0.1676 - acc: 0.9656
Epoch 181/250
 - 0s - loss: 0.1667 - acc: 0.9663
Epoch 182/250
 - 0s - loss: 0.1659 - acc: 0.9663
Epoch 183/250
 - 0s - loss: 0.1650 - acc: 0.9663
Epoch 184/250
 - 0s - loss: 0.1642 - acc: 0.9675
Epoch 185/250
 - 0s - loss: 0.1634 - acc: 0.9675
Epoch 186/250
 - 0s - loss: 0.1626 - acc: 0.9675
Epoch 187/250
 - 0s - loss: 0.1618 - acc: 0.9675
Epoch 188/250
 - 0s - loss: 0.1610 - acc: 0.9675
Epoch 189/250
 - 0s - loss: 0.1602 - acc: 0.9675
Epoch 190/250
 - 0s - loss: 0.1594 - acc: 0.9675
Epoch 191/250
 - 0s 

Por último, podemos utilizar la siguiente función para calcular la precisión que obtenemos con la red neuronal construida utilizando únicamente numpy.

In [18]:
def accuracy(X, y, parameters):
    """
    Calcula la precisión de las predicciones de la red neuronal.
    
    Argumentos:
    X -- datos: matriz de tamaño (número de variables, número de ejemplos)
    parameters -- parámetros de la red neuronal entrenada
    
    Returns:
    accuracy -- valor entre 0 y 1 que representa la precisión de la red neuronal
    """
    
    m = X.shape[1]
    p = np.zeros((1,m))
    
    # Propagación hacia delante
    probas, caches = L_model_forward(X, parameters)

    # Conversión de la salida de la red a valores 0 o 1
    for i in range(0, probas.shape[1]):
        if probas[0, i] > 0.5:
            p[0, i] = 1
        else:
            p[0, i] = 0
            
    accuracy = np.sum((p == y)) / m
    
    return accuracy

A continuación se muestra la precisión obtenida tanto con la red construida con numpy como con la red construida con Keras.

In [19]:
print("Red construida con numpy")
print("Precisión {:.2f}".format(accuracy(test_x.T, test_y.reshape(1, -1), parameters)))
print("---")
print("Red construida con Keras")
print("Precisión {:.2f}".format(model.evaluate(test_x, test_y, verbose=0)[1]))

Red construida con numpy
Precisión 0.49
---
Red construida con Keras
Precisión 0.99


El siguiente código permite calcular el tiempo que tarda cada red neuronal en entrenarse.

In [20]:
%%timeit
parameters = L_layer_model(train_x.T, train_y.reshape(1, -1), layers_dims=layers_dims, 
                           learning_rate=0.1, num_iterations=250, print_cost=False)

385 ms ± 2.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
model = keras_model(layers_dims=layers_dims, learning_rate=0.1)
model.fit(train_x, train_y, epochs=250, batch_size=train_x.shape[0], verbose=0)

1.02 s ± 51.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Análisis:</strong> Comparar el rendimiento, tanto en tiempo de ejecución como en precisión, de las dos implementaciones de la red neuronal. Utilizar diferentes hiperparámetros en la comparación: probar con diferentes valores para las dimensiones de las capas, diferente número de capas, número de iteraciones, etc. ¿Qué factores pueden estar creando las diferencias observadas?
</div>

Antes de empezar con las comprobaciones con hiperparámetros, algo que queda claro es que la **precisión** de la red construida usando **Keras** es significativamente **mejor** que nuestra implementación con **numpy**, aunque el **tiempo para entrenarse es más grande en el primer**.

In [22]:
#parameters = L_layer_model(train_x.T, train_y.reshape(1, -1), layers_dims=layers_dims, learning_rate=9, 
#                           num_iterations=1000, print_cost=True)

In [23]:
print("Precisión {:.2f}".format(accuracy(test_x.T, test_y.reshape(1, -1), parameters)))

Precisión 0.49


### References
Parte del codigo utilizado para desarrollar esta práctica proviene del curso de Coursera ["Neural networks and deep learning"](https://www.coursera.org/learn/neural-networks-deep-learning?specialization=deep-learning)